# 2. 02_silver_transformations — Camada Silver (Clean / Curated)

## 2.1 Objetivo deste notebook
Este notebook transforma a tabela **Bronze** em uma tabela **Silver**, com:
- tipagem e padronização de atributos;
- criação de campos derivados;
- extração de atributos do PGN (ex.: Date/UTCDate, ECO, Termination, Link);
- validações de qualidade (domínios, nulos, duplicidade).

## 2.2 Regras principais
- `rating_diff = white_rating - black_rating`
- `outcome`:
  - `white_win` se `white_result = "win"`
  - `black_win` se `black_result = "win"`
  - `draw` se ambos os lados indicarem motivo de empate (ex.: stalemate, repetition, agreed, insufficient, 50move, timevsinsufficient)
  - `unknown` caso contrário (para evidenciar inconsistências de domínio)
- `score`: vitória = 1, empate = 0.5, derrota = 0

## 2.3 Evidências
Este notebook apresenta:
- contagem antes/depois de deduplicação;
- schema e amostra da Silver;
- checagens de nulos, domínios e duplicidade (por `game_key`).


## 2.4 Importar Bronze e definir datatypes

In [0]:
from pyspark.sql import functions as F

# Bronze Table
bronze_table = "workspace.default.bronze_games"
df_bronze = spark.table(bronze_table)
print("Bronze table:", bronze_table)

# Definir Datatypes
type_map = {
    "white_rating": "int",
    "black_rating": "int",
    "rated": "boolean",
    "white_id": "string",
    "black_id": "string",
    "time_class": "string",
    "time_control": "string",
    "rules": "string",
    "white_result": "string",
    "black_result": "string",
    "pgn": "string"
}

df_silver = df_bronze
for col_name, spark_type in type_map.items():
    if col_name in df_silver.columns:
        df_silver = df_silver.withColumn(col_name, F.col(col_name).cast(spark_type))

display(df_silver.limit(10))
df_silver.printSchema()


## 2.5 Extrair dados da coluna PGN

In [0]:
# Função auxiliar para extrair tags do PGN (ex.: [ECO "B01"])
def pgn_tag(col_pgn, tag_name: str):
    return F.when(
        col_pgn.isNotNull(),
        F.regexp_extract(col_pgn, rf'\[{tag_name} "([^"]+)"\]', 1)
    ).otherwise(F.lit(None).cast("string"))


# Extrair tags principais do PGN
df_silver = (
    df_silver
    .withColumn("pgn_utc_date", pgn_tag(F.col("pgn"), "UTCDate"))
    .withColumn("pgn_date", pgn_tag(F.col("pgn"), "Date"))
    .withColumn("pgn_eco", pgn_tag(F.col("pgn"), "ECO"))
    .withColumn("pgn_eco_url", pgn_tag(F.col("pgn"), "ECOUrl"))
    .withColumn("pgn_termination", pgn_tag(F.col("pgn"), "Termination"))
    .withColumn("pgn_link", pgn_tag(F.col("pgn"), "Link"))
)

# Derivar game_date: prefer UTCDate, senão Date (formato comum: YYYY.MM.DD)
df_silver = (
    df_silver
    .withColumn(
        "game_date_raw",
        F.when(F.length(F.col("pgn_utc_date")) > 0, F.col("pgn_utc_date"))
         .when(F.length(F.col("pgn_date")) > 0, F.col("pgn_date"))
         .otherwise(F.lit(None).cast("string"))
    )
    .withColumn("game_date", F.to_date(F.regexp_replace(F.col("game_date_raw"), r"\.", "-"), "yyyy-MM-dd"))
)

print("Novas colunas:")
display(df_silver.select("pgn_utc_date","pgn_date","game_date_raw","game_date","pgn_eco","pgn_termination","pgn_link").limit(10))

## 2.6 Derivar colunas/atributos

In [0]:
draw_reasons = ["stalemate", "repetition", "agreed", "insufficient", "50move", "timevsinsufficient"]

df_silver = (
    df_silver
    .withColumn("rating_diff", F.col("white_rating") - F.col("black_rating"))
    .withColumn(
        "outcome",
        F.when(F.col("white_result") == F.lit("win"), F.lit("white_win"))
         .when(F.col("black_result") == F.lit("win"), F.lit("black_win"))
         .when(
             F.col("white_result").isin(draw_reasons) & F.col("black_result").isin(draw_reasons),
             F.lit("draw")
         )
         .otherwise(F.lit("unknown"))
    )
    .withColumn(
        "white_score",
        F.when(F.col("outcome") == "white_win", F.lit(1.0))
         .when(F.col("outcome") == "draw", F.lit(0.5))
         .when(F.col("outcome") == "black_win", F.lit(0.0))
         .otherwise(F.lit(None).cast("double"))
    )
    .withColumn(
        "black_score",
        F.when(F.col("outcome") == "black_win", F.lit(1.0))
         .when(F.col("outcome") == "draw", F.lit(0.5))
         .when(F.col("outcome") == "white_win", F.lit(0.0))
         .otherwise(F.lit(None).cast("double"))
    )
)

# termination_type: para vitórias, o motivo aparece no "result" do perdedor.
# Para draws/unknown, está em pgn_termination quando disponível. Fallback nos campos de resultado.
df_silver = (
    df_silver
    .withColumn(
        "termination_type",
        F.when(F.col("outcome") == "white_win", F.col("black_result"))
         .when(F.col("outcome") == "black_win", F.col("white_result"))
         .otherwise(F.col("pgn_termination"))
    )
    .withColumn(
        "termination_type",
        F.when(F.length(F.col("termination_type")) > 0, F.col("termination_type"))
         .otherwise(F.col("white_result"))
    )
)

display(df_silver.select("white_result","black_result","outcome","white_score","black_score","rating_diff","termination_type").limit(20))


## 2.7 Criar Game Key (para dedupe)

In [0]:
# Construir game_key com fallback (só usa colunas que existirem)
candidate_cols = []
for c in ["pgn_link", "game_url", "url", "link"]:   # o dataset pode variar
    if c in df_silver.columns:
        candidate_cols.append(F.col(c))

if candidate_cols:
    game_key = F.coalesce(*candidate_cols)
else:
    # fallback extremo: hash do PGN (se existir) ou concat de ids+time
    if "pgn" in df_silver.columns:
        game_key = F.sha2(F.col("pgn"), 256)
    else:
        game_key = F.sha2(F.concat_ws("||", F.col("white_id").cast("string"), F.col("black_id").cast("string"), F.col("time_control").cast("string")), 256)

df_silver = df_silver.withColumn("game_key", game_key)

n_before = df_silver.count()
df_silver_dedup = df_silver.dropDuplicates(["game_key"])
n_after = df_silver_dedup.count()

print("Linhas antes do dedup:", n_before)
print("Linhas depois do dedup:", n_after)
print("Dedup removeu:", n_before - n_after)

# Checks: domínios e nulos (principais)
display(
    df_silver_dedup.groupBy("outcome").count().orderBy(F.desc("count"))
)

display(
    df_silver_dedup.groupBy("time_class").count().orderBy(F.desc("count"))
)

# Null checks (campos principais)
qc_cols = ["white_rating", "black_rating", "white_result", "black_result", "time_class", "rated", "pgn_eco", "termination_type", "game_key"]
qc_exprs = []
for c in qc_cols:
    if c in df_silver_dedup.columns:
        qc_exprs.append(F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"null_{c}"))

display(df_silver_dedup.agg(*qc_exprs))


## 2.8 Salvar a Silver Table

In [0]:
silver_table = "workspace.default.silver_games"

(df_silver_dedup.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_table)
)

print("Silver salvo em:", silver_table)


## SQL Queries de teste

In [0]:
%sql
SELECT COUNT(*) AS n_rows FROM workspace.default.silver_games;


In [0]:
%sql
SELECT outcome, COUNT(*) AS n
FROM workspace.default.silver_games
GROUP BY outcome
ORDER BY n DESC;


In [0]:
%sql
SELECT *
FROM workspace.default.silver_games
LIMIT 10;
